In [1]:
import pandas as pd
import earthaccess
import geopandas as gpd
import xarray as xr
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import shape
import time
import folium

floripa_bounds = (-48.7242431, -27.917059, -48.3585929, -27.3804967)
floripa_center = [-27.5935, -48.5585]

def get_districts_loction(df_districts):
    geolocator = Nominatim(user_agent="busca_bairros_floripa_app")
    geometrias = []

    print(f"Iniciando a busca de geometrias para {len(df_districts)} bairros. Isso pode levar um minuto...")

    for bairro in df_districts['Bairro']:
        try:
            query = f"{bairro}, Florianópolis, Santa Catarina"
            location = geolocator.geocode(query, geometry='geojson')
            
            if location and 'geojson' in location.raw:
                geom = shape(location.raw['geojson'])
                geometrias.append(geom)
            else:
                geometrias.append(None)
        except Exception as e:
            geometrias.append(None)
        time.sleep(1)

    df_districts['geometry'] = geometrias
    bairros_gdf = gpd.GeoDataFrame(df_districts, geometry='geometry', crs="EPSG:4236")
    bairros_gdf.dropna(subset=['geometry'], inplace=True)

    return bairros_gdf

def download_metric(metric, range_dates, bbox, files_qty=1):
    try:
        auth = earthaccess.login(strategy="netrc")
        if not auth.authenticated:
            print("Autenticação falhou! Verifique seu arquivo .netrc.")
            return
    except Exception as e:
        print(f"Erro na autenticação: {e}")
        return
    
    # Procurando os dados GPM IMERG Half-Hourly
    # Short Name para o produto IMERG Final Run Half-Hourly V07: GPM_3IMERGHH
    results = earthaccess.search_data(
        short_name=metric,
        bounding_box=bbox,
        temporal=range_dates, # Exemplo: buscando 1 dia de dados
        count=files_qty
    )
    if not results:
        print("pesquisa sem resultados")
        return
    
    data_files = earthaccess.download(results[0], local_path="data")
    return data_files

def merge_with_precipitation(file_path: str, df_districts):
    # Abrir o arquivo HDF5 usando xarray
    # O grupo 'Grid' geralmente contém os dados de precipitação
    ds = xr.open_dataset(file_path, group='Grid')
    
    # Extrair as variáveis de interesse
    # 'precipitationCal' é a precipitação calibrada, a mais recomendada
    precip_data = ds['precipitation'].values[0] # Pegando o primeiro slice de tempo
    lats = ds['lat'].values
    lons = ds['lon'].values

    # Os dados de lon/lat do GPM formam uma grade (grid).
    # Precisamos criar um GeoDataFrame com cada ponto dessa grade.
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    points = [Point(lon, lat) for lon, lat in zip(lon_grid.ravel(), lat_grid.ravel())]
    precip_values = precip_data.ravel()

    gpm_gdf = gpd.GeoDataFrame({
        'precipitation_mmhr': precip_values,
        'geometry': points
    }, crs="EPSG:4326")

    df_districts = gpd.GeoDataFrame(df_districts, geometry='geometry', crs="EPSG:4326")
    bairros_com_chuva = gpd.sjoin(df_districts, gpm_gdf, how="left", predicate="contains")
    
    resultado_final = bairros_com_chuva.drop(columns=["index_right"])
    resultado_final = resultado_final.replace(np.nan, 0)
    
    return resultado_final


def merge_with_smap(file_path: str, bairros_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    ds = xr.open_dataset(file_path)
    data_group = ds['Soil_Moisture_Retrieval_Data_AM']
    moisture_variable = data_group['soil_moisture']
    moisture_data = moisture_variable.values
    lats = data_group['latitude'].values
    lons = data_group['longitude'].values

    moisture_values_flat = moisture_data.ravel()
    lons_flat = lons.ravel()
    lats_flat = lats.ravel()

    fill_value = moisture_variable.attrs.get('_FillValue', -9999.0)
    valid_mask = moisture_values_flat != fill_value

    points = [Point(lon, lat) for lon, lat in zip(lons_flat[valid_mask], lats_flat[valid_mask])]
    valid_moisture_values = moisture_values_flat[valid_mask]

    smap_gdf = gpd.GeoDataFrame({
        'soil_moisture': valid_moisture_values,
        'geometry': points
    }, crs="EPSG:4326")

    bairros_com_smap = gpd.sjoin(bairros_gdf, smap_gdf, how="left", predicate="contains")

    if 'soil_moisture' in bairros_com_smap.columns:
        smap_media_por_bairro = bairros_com_smap.groupby('Bairro')['soil_moisture'].mean()

        resultado_final = bairros_gdf.merge(
            smap_media_por_bairro,
            on='Bairro',
            how='left'
        )
        resultado_final['soil_moisture'].fillna(0, inplace=True)
    else:
        resultado_final = bairros_gdf.copy()
        resultado_final['soil_moisture'] = 0

    return pd.DataFrame(resultado_final)

def generate_html_graph(data, center, map_name):
    m = folium.Map(location=center, zoom_start=11)

    folium.GeoJson(
        data,
        name=map_name,
        tooltip=folium.GeoJsonTooltip(fields=['Bairro'], aliases=['Bairro:']),
        style_function=lambda x: {
            'fillColor': '#4CAF50', # Cor de preenchimento verde
            'color': 'black',      # Cor da borda preta
            'weight': 1,           # Espessura da borda
            'fillOpacity': 0.4     # Opacidade do preenchimento
        }
    ).add_to(m)

    folium.LayerControl().add_to(m)

    mapa_path = f'{map_name}.html'
    m.save(mapa_path)
    

d:\Users\Bee\Estudos\nasa-challenge\nasa-challenge\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
date = ("2025-01-30", "2025-01-31")

df_floripa_districts = pd.read_csv(r"data\floripa_districts.csv")
df_floripa_districts = get_districts_loction(df_floripa_districts)

gpm = download_metric("GPM_3IMERGHHL", date, floripa_bounds)
df_floripa_districts = merge_with_precipitation(gpm[0], df_floripa_districts)

Iniciando a busca de geometrias para 49 bairros. Isso pode levar um minuto...


QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.81s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]


In [ ]:
smap = download_metric("SPL3SMP_E", date, floripa_bounds)
df_floripa_districts = merge_with_precipitation(smap[0], df_floripa_districts)

QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 999.83it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [01:01<00:00, 61.87s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]


In [ ]:

f_r = merge_with_precipitation(smap[0], df_floripa_districts)

,Bairro,geometry
0,Cachoeira do Bom Jesus,"POLYGON ((-48.45818 -27.46639, -48.4577 -27.46..."
1,Canasvieiras,"POLYGON ((-48.54722 -27.45903, -48.54715 -27.4..."
2,Daniela,"POLYGON ((-48.54722 -27.45903, -48.54715 -27.4..."
3,Ingleses do Rio Vermelho,"POLYGON ((-48.41971 -27.45198, -48.41971 -27.4..."
4,Jurerê Internacional,"POLYGON ((-48.52374 -27.45185, -48.52362 -27.4..."
5,Jurerê,"POLYGON ((-48.51496 -27.43202, -48.51487 -27.4..."
6,Ponta das Canas,"POLYGON ((-48.43663 -27.39291, -48.43661 -27.3..."
7,Praia Brava,"POLYGON ((-48.41971 -27.38264, -48.41734 -27.3..."
8,Sambaqui,"POLYGON ((-48.53954 -27.48683, -48.53953 -27.4..."
9,Santo Antônio de Lisboa,"POLYGON ((-48.53954 -27.48683, -48.53953 -27.4..."
